In [2]:
import numpy as np
import pandas as pd
import sqlite3
import csv
import os
import glob

In [ ]:
import pandas as pd
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas

# Connect to Snowflake
conn = snowflake.connector.connect(
    user='******',
    password='********',
    account='**********',
    warehouse='COMPUTE_WH',
    database='TRANING',
    schema='PUBLIC'
)

cursor = conn.cursor()


In [ ]:
df = pd.read_csv(r"C:\Users\Desktop\EPL Data.csv")


In [7]:
df.head()

SEASON  GAME_WEEK  MATCH_NO       HOME_TEAM    AWAY_TEAM  HOME_SCORE  \
0  2014-2015          1         1         Man Utd      Swansea           1   
1  2014-2015          1         2  Leicester City      Everton           2   
2  2014-2015          1         3        West Ham    Tottenham           0   
3  2014-2015          1         4             QPR    Hull City           0   
4  2014-2015          1         5           Stoke  Aston Villa           0   

   AWAY_SCORE  
0           2  
1           2  
2           1  
3           1  
4           1

In [6]:
df.columns = df.columns.str.strip().str.replace(" ", "_").str.upper()

In [57]:
# Automatically creates the table based on DataFrame columns/types
success, nchunks, nrows, _ = write_pandas(
    conn, df, 'EPL', auto_create_table=True, overwrite=True
)
print(f"Uploaded {nrows} rows into EPL")


Uploaded 2660 rows into EPL


In [58]:
df_count = len(df)
cursor.execute(f"SELECT COUNT(*) FROM EPL")
snow_count = cursor.fetchone()[0]
print(f"Rows in DataFrame: {df_count}, Rows in Snowflake: {snow_count}")


Rows in DataFrame: 2660, Rows in Snowflake: 2660


In [59]:
print("Pandas head():")
print(df.head())
print("\nSnowflake sample:")
cursor.execute(f"SELECT * FROM EPL LIMIT 5")
print(cursor.fetchall())


Pandas head():
      SEASON  GAME_WEEK  MATCH_NO       HOME_TEAM    AWAY_TEAM  HOME_SCORE  \
0  2014-2015          1         1         Man Utd      Swansea           1   
1  2014-2015          1         2  Leicester City      Everton           2   
2  2014-2015          1         3        West Ham    Tottenham           0   
3  2014-2015          1         4             QPR    Hull City           0   
4  2014-2015          1         5           Stoke  Aston Villa           0   

   AWAY_SCORE  
0           2  
1           2  
2           1  
3           1  
4           1  

Snowflake sample:
[('2014-2015', 1, 1, 'Man Utd', 'Swansea', 1, 2), ('2014-2015', 1, 2, 'Leicester City', 'Everton', 2, 2), ('2014-2015', 1, 3, 'West Ham', 'Tottenham', 0, 1), ('2014-2015', 1, 4, 'QPR', 'Hull City', 0, 1), ('2014-2015', 1, 5, 'Stoke', 'Aston Villa', 0, 1)]


In [37]:
def run_query(sql):
    return pd.read_sql(sql, conn)

In [67]:
# Get column names and datatypes from Snowflake
df_desc = run_query("DESC TABLE EPL;")
df_desc[['name', 'type']]


name               type
0      SEASON  VARCHAR(16777216)
1   GAME_WEEK       NUMBER(38,0)
2    MATCH_NO       NUMBER(38,0)
3   HOME_TEAM  VARCHAR(16777216)
4   AWAY_TEAM  VARCHAR(16777216)
5  HOME_SCORE       NUMBER(38,0)
6  AWAY_SCORE       NUMBER(38,0)

In [66]:
q1 = """
SELECT SEASON, COUNT(*) AS DRAWN_GAMES
FROM EPL
WHERE HOME_SCORE = AWAY_SCORE
GROUP BY SEASON
ORDER BY SEASON;
"""
df1 = run_query(q1)
print(df1)


      SEASON  DRAWN_GAMES
0  2014-2015           93
1  2015-2016          107
2  2016-2017           84
3  2017-2018           99
4  2018-2019           71
5  2019-2020           92
6  2020-2021           83


In [65]:
q2 = """
WITH team_points AS (
    SELECT SEASON, HOME_TEAM AS TEAM,
           SUM(CASE WHEN HOME_SCORE > AWAY_SCORE THEN 3
                    WHEN HOME_SCORE = AWAY_SCORE THEN 1 ELSE 0 END) AS PTS
    FROM EPL
    GROUP BY SEASON, HOME_TEAM
    UNION ALL
    SELECT SEASON, AWAY_TEAM AS TEAM,
           SUM(CASE WHEN AWAY_SCORE > HOME_SCORE THEN 3
                    WHEN HOME_SCORE = AWAY_SCORE THEN 1 ELSE 0 END) AS PTS
    FROM EPL
    GROUP BY SEASON, AWAY_TEAM
),
total_points AS (
    SELECT SEASON, TEAM, SUM(PTS) AS POINTS
    FROM team_points
    GROUP BY SEASON, TEAM
)
SELECT SEASON, TEAM, POINTS
FROM total_points
QUALIFY RANK() OVER (PARTITION BY SEASON ORDER BY POINTS DESC) = 1;
"""
df2 = run_query(q2)
print(df2)


      SEASON            TEAM  POINTS
0  2014-2015         Chelsea      87
1  2017-2018        Man City     100
2  2018-2019        Man City      98
3  2019-2020       Liverpool      99
4  2016-2017         Chelsea      93
5  2015-2016  Leicester City      81
6  2020-2021        Man City      86


In [68]:
q3 = """
WITH goals AS (
    SELECT SEASON, HOME_TEAM AS TEAM, SUM(HOME_SCORE) AS GOALS
    FROM EPL
    GROUP BY SEASON, HOME_TEAM
    UNION ALL
    SELECT SEASON, AWAY_TEAM AS TEAM, SUM(AWAY_SCORE) AS GOALS
    FROM EPL
    GROUP BY SEASON, AWAY_TEAM
),
total_goals AS (
    SELECT SEASON, TEAM, SUM(GOALS) AS GOALS_SCORED
    FROM goals
    GROUP BY SEASON, TEAM
)
SELECT *
FROM total_goals
QUALIFY RANK() OVER (PARTITION BY SEASON ORDER BY GOALS_SCORED DESC) = 1;
"""
df3 = run_query(q3)
print(df3)


      SEASON       TEAM  GOALS_SCORED
0  2014-2015   Man City            83
1  2017-2018   Man City           106
2  2018-2019   Man City            95
3  2019-2020   Man City           102
4  2016-2017  Tottenham            86
5  2015-2016   Man City            71
6  2020-2021   Man City            83


In [73]:
q4 = """
WITH conceded AS (
    SELECT SEASON, HOME_TEAM AS TEAM, SUM(AWAY_SCORE) AS GOALS_CONCEDED
    FROM EPL
    GROUP BY SEASON, HOME_TEAM
    UNION ALL
    SELECT SEASON, AWAY_TEAM AS TEAM, SUM(HOME_SCORE) AS GOALS_CONCEDED
    FROM EPL
    GROUP BY SEASON, AWAY_TEAM
),
total_conceded AS (
    SELECT SEASON, TEAM, SUM(GOALS_CONCEDED) AS GOALS_CONCEDED
    FROM conceded
    GROUP BY SEASON, TEAM
)
SELECT *
FROM total_conceded
QUALIFY RANK() OVER (PARTITION BY SEASON ORDER BY GOALS_CONCEDED ASC) = 1;
"""
df4 = run_query(q4)
print(df4)


      SEASON       TEAM  GOALS_CONCEDED
0  2014-2015    Chelsea              32
1  2017-2018   Man City              27
2  2018-2019  Liverpool              22
3  2019-2020  Liverpool              33
4  2016-2017  Tottenham              26
5  2015-2016  Tottenham              35
6  2015-2016    Man Utd              35
7  2020-2021   Man City              32


In [76]:
conn.close()
